In [1]:
import pandas as pd
import polars as pl
import datetime as dt
from sqlalchemy import create_engine, inspect, text
import os

In [2]:
engine = create_engine(os.environ["REDSHIFT_READ_URI"], echo=False)

def sql_df(query):
  with engine.connect() as conexao:
    consulta = conexao.execute(text(query))
    dados = consulta.fetchall()
  df = pd.DataFrame(dados, columns= consulta.keys())
  return df

In [18]:
query = ''' 

select 
deal_id,
c.customer_id,
customer_document,
d.days_of_delay base
from consolidacao_carteira.deal_39 d
left join consolidacao_carteira.customer_39 c on d.deal_id = c.best_deal_id 
where d.days_of_delay >= 75 and is_hot is true
order by days_of_delay  

'''


In [14]:
consulta = sql_df(query)

In [20]:
consulta['type'] = 'CLUSTER'
consulta['key'] = 'TESTE_CLUSTER'
consulta['creditor_id'] = 1 
consulta['expire_at'] = '2024-08-15 00:00:00.000'

# Para Exportar em CSV

In [17]:
nome_aquivo = 'Base_Digio'
consulta.to_csv(f'{nome_aquivo}.csv', index=False)